In [12]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
physical_devices = tf.config.list_physical_devices('GPU')
try:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
except:
    # Invalid device or cannot modify virtual devices once initi
    # alized.
    print('Could not initialize the tensorflow gpu')
    pass

In [18]:
identifiers = np.loadtxt("labels50.txt", dtype="str")
features = np.loadtxt("features50.txt")
labels = np.take(identifiers,2, axis=1)

In [20]:
labels = labels.astype(int)

In [14]:
features[0][0:6]

array([   7.38043478,    6.9673913 ,   -5.08695652, -205.73170732,
         39.14634146, -153.01829268])

In [87]:
features = features.reshape(4272,50,6)

In [89]:
input = layers.Input(shape = (50,6))
#model = layers.Conv1D(50, kernel_size = 6, activation='relu')(input)
model = layers.LSTM(100)(input)
model = layers.GlobalAveragePooling1D()(model)
#model = layers.Flatten()(model)
model = layers.Dense(100)(model)
output = layers.Dense(1, 'sigmoid')(model)
model = tf.keras.Model(inputs=[input], outputs=output)
optimizer = tf.keras.optimizers.Adam(1e-3)
loss = tf.keras.losses.BinaryCrossentropy()
model.compile(loss = loss, optimizer=optimizer, metrics = ['accuracy'])

ValueError: Input 0 of layer global_average_pooling1d_12 is incompatible with the layer: expected ndim=3, found ndim=2. Full shape received: (None, 100)

In [60]:
X_Train.shape

(3417, 300)

In [61]:
hist = model.fit(X_Train.reshape(len(X_Train), 50,6),y_Train, epochs=100, shuffle=True,validation_split=0.2)

Epoch 1/100
86/86 [==============================] - 1s 10ms/step - loss: 3.1640 - accuracy: 0.5540 - val_loss: 0.8145 - val_accuracy: 0.6126
Epoch 2/100
86/86 [==============================] - 0s 2ms/step - loss: 1.0513 - accuracy: 0.6058 - val_loss: 1.0144 - val_accuracy: 0.5980
Epoch 3/100
86/86 [==============================] - 0s 1ms/step - loss: 0.9418 - accuracy: 0.6120 - val_loss: 0.8665 - val_accuracy: 0.6389
Epoch 4/100
86/86 [==============================] - 0s 1ms/step - loss: 0.8962 - accuracy: 0.6155 - val_loss: 0.8179 - val_accuracy: 0.6243
Epoch 5/100
86/86 [==============================] - 0s 1ms/step - loss: 0.7914 - accuracy: 0.6612 - val_loss: 0.6726 - val_accuracy: 0.6564
Epoch 6/100
86/86 [==============================] - 0s 1ms/step - loss: 0.8701 - accuracy: 0.6078 - val_loss: 0.7113 - val_accuracy: 0.6740
Epoch 7/100
86/86 [==============================] - 0s 1ms/step - loss: 0.6969 - accuracy: 0.6631 - val_loss: 0.7131 - val_accuracy: 0.6491
Epoch 8/100


In [63]:
preds = [1 if p > 0.5 else 0 for p in model.predict(X_Test.reshape(len(X_Test), 50,6))]
from sklearn.metrics import accuracy_score, classification_report

accuracy_score(y_Test,preds)

0.7461988304093568

In [82]:
# from collections import Counter 

# Counter(preds)
# print(classification_report(labels, preds))


In [65]:
model.summary( )

Model: "model_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         [(None, 50, 6)]           0         
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 45, 50)            1850      
_________________________________________________________________
global_average_pooling1d_8 ( (None, 50)                0         
_________________________________________________________________
dense_11 (Dense)             (None, 100)               5100      
_________________________________________________________________
dense_12 (Dense)             (None, 1)                 101       
Total params: 7,051
Trainable params: 7,051
Non-trainable params: 0
_________________________________________________________________


In [66]:
flatFeatures = features.flatten().reshape(4272,300)

In [81]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

X_Train, X_Test, y_Train, y_Test = train_test_split(flatFeatures, labels, random_state=43, test_size=0.2, shuffle=True)

rf = RandomForestClassifier(n_estimators=1)

rf.fit(X_Train, y_Train)

rf.score(X_Test,y_Test)

0.7076023391812866